In [ ]:
import pandas as pd
import numpy as np


song_data = pd.read_csv('song_data.csv.zip') 
song_info = pd.read_csv('song_info.csv.zip')

In [ ]:
song_data.head()

,song_name,song_popularity,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence
0,Boulevard of Broken Dreams,73,262333,0.005520,0.496,0.682,0.000029,8,0.0589,-4.095,1,0.0294,167.060,4,0.474
1,In The End,66,216933,0.010300,0.542,0.853,0.000000,3,0.1080,-6.407,0,0.0498,105.256,4,0.370
2,Seven Nation Army,76,231733,0.008170,0.737,0.463,0.447000,0,0.2550,-7.828,1,0.0792,123.881,4,0.324
3,By The Way,74,216933,0.026400,0.451,0.970,0.003550,0,0.1020,-4.938,1,0.1070,122.444,4,0.198
4,How You Remind Me,56,223826,0.000954,0.447,0.766,0.000000,10,0.1130,-5.065,1,0.0313,172.011,4,0.574


In [ ]:
song_info.head()

,song_name,artist_name,album_names,playlist
0,Boulevard of Broken Dreams,Green Day,Greatest Hits: God's Favorite Band,00s Rock Anthems
1,In The End,Linkin Park,Hybrid Theory,00s Rock Anthems
2,Seven Nation Army,The White Stripes,Elephant,00s Rock Anthems
3,By The Way,Red Hot Chili Peppers,By The Way (Deluxe Version),00s Rock Anthems
4,How You Remind Me,Nickelback,Silver Side Up,00s Rock Anthems


In [ ]:
song_info[song_info['song_name'] == 'Footloose']

,song_name,artist_name,album_names,playlist
778,Footloose,Kenny Loggins,Footloose (15th Anniversary Collectors' Edition),80s Smash Hits
2243,Footloose,Kenny Loggins,The Essential Kenny Loggins,All Out 80s
6403,Footloose,Kenny Loggins,"Yesterday, Today, Tomorrow - The Greatest Hits...",Energizing Classics
15934,Footloose,Kenny Loggins,Footloose (15th Anniversary Collectors' Edition),Songs to Sing in the Shower


**Ubah semua string menjadi huruf kecil**

In [ ]:
song_info['playlist'] = song_info['playlist'].str.lower()
song_info['song_name'] = song_info['song_name'].str.lower()
song_info['artist_name'] = song_info['artist_name'].str.lower()
song_info['album_names'] = song_info['album_names'].str.lower()

song_data['song_name'] = song_data['song_name'].str.lower()



#song_info[song_info['playlist'].str.contains('pop')]

**Menggabungkan dua kumpulan data menjadi satu**

In [ ]:
song_df = song_data.copy()
song_df['artist_name'] = song_info['artist_name']


song_df = song_df.drop_duplicates(subset=['song_name','artist_name'])

song_info = song_info.drop_duplicates(subset=['song_name','artist_name'])


song_df = song_df.reset_index()
song_df = song_df.drop('index',axis=1)

song_df.head()

,song_name,song_popularity,song_duration_ms,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,time_signature,audio_valence,artist_name
0,boulevard of broken dreams,73,262333,0.005520,0.496,0.682,0.000029,8,0.0589,-4.095,1,0.0294,167.060,4,0.474,green day
1,in the end,66,216933,0.010300,0.542,0.853,0.000000,3,0.1080,-6.407,0,0.0498,105.256,4,0.370,linkin park
2,seven nation army,76,231733,0.008170,0.737,0.463,0.447000,0,0.2550,-7.828,1,0.0792,123.881,4,0.324,the white stripes
3,by the way,74,216933,0.026400,0.451,0.970,0.003550,0,0.1020,-4.938,1,0.1070,122.444,4,0.198,red hot chili peppers
4,how you remind me,56,223826,0.000954,0.447,0.766,0.000000,10,0.1130,-5.065,1,0.0313,172.011,4,0.574,nickelback


**periksa apakah fitur numerik mengikuti distribusi Gaussian**

In [ ]:
from scipy.stats import shapiro

stat,p =shapiro(song_df[['song_popularity', 'acousticness',
       'danceability', 'energy', 'instrumentalness', 'key', 'liveness',
       'loudness', 'audio_mode', 'speechiness', 'tempo','audio_valence']])

print("stat: ",stat,":","p :",p)
if p>0.05:
    print("gaussian distribution - fail to reject H0")
else:
    print("not gaussian distribution - reject H0")

stat:  0.5140853524208069 : p : 0.0
not gaussian distribution - reject H0


/usr/local/lib/python3.7/dist-packages/scipy/stats/morestats.py:1676: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


In [ ]:
song_df = song_df.drop(['time_signature','song_duration_ms'], axis=1)
song_df.head()

,song_name,song_popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,audio_valence,artist_name
0,boulevard of broken dreams,73,0.005520,0.496,0.682,0.000029,8,0.0589,-4.095,1,0.0294,167.060,0.474,green day
1,in the end,66,0.010300,0.542,0.853,0.000000,3,0.1080,-6.407,0,0.0498,105.256,0.370,linkin park
2,seven nation army,76,0.008170,0.737,0.463,0.447000,0,0.2550,-7.828,1,0.0792,123.881,0.324,the white stripes
3,by the way,74,0.026400,0.451,0.970,0.003550,0,0.1020,-4.938,1,0.1070,122.444,0.198,red hot chili peppers
4,how you remind me,56,0.000954,0.447,0.766,0.000000,10,0.1130,-5.065,1,0.0313,172.011,0.574,nickelback


**Standarisasi menggunakan MinMaxScaler**

In [ ]:
from sklearn.preprocessing import MinMaxScaler


minmaxscaler = MinMaxScaler()
minmaxscaled = minmaxscaler.fit_transform(song_df[['song_popularity', 'acousticness',
       'danceability', 'energy', 'instrumentalness', 'key', 'liveness',
       'loudness', 'audio_mode', 'speechiness', 'tempo','audio_valence']])
songs_normalized = pd.DataFrame(minmaxscaled, columns=[['song_popularity', 'acousticness',
       'danceability', 'energy', 'instrumentalness', 'key', 'liveness',
       'loudness', 'audio_mode', 'speechiness', 'tempo','audio_valence']])

In [ ]:
song_df_normalized = song_df.copy()
song_df_normalized.head()

,song_name,song_popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,audio_valence,artist_name
0,boulevard of broken dreams,73,0.005520,0.496,0.682,0.000029,8,0.0589,-4.095,1,0.0294,167.060,0.474,green day
1,in the end,66,0.010300,0.542,0.853,0.000000,3,0.1080,-6.407,0,0.0498,105.256,0.370,linkin park
2,seven nation army,76,0.008170,0.737,0.463,0.447000,0,0.2550,-7.828,1,0.0792,123.881,0.324,the white stripes
3,by the way,74,0.026400,0.451,0.970,0.003550,0,0.1020,-4.938,1,0.1070,122.444,0.198,red hot chili peppers
4,how you remind me,56,0.000954,0.447,0.766,0.000000,10,0.1130,-5.065,1,0.0313,172.011,0.574,nickelback


In [ ]:
song_df_normalized[['song_popularity', 'acousticness',
       'danceability', 'energy', 'instrumentalness', 'key', 'liveness',
       'loudness', 'audio_mode', 'speechiness', 'tempo','audio_valence']] = songs_normalized
#song_df_normalized.drop(['bpm','nrgy','dnce','dB','live','val','acous','spch','pop'],axis=1,inplace=True)
song_df_normalized.head()

,song_name,song_popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,audio_valence,artist_name
0,boulevard of broken dreams,0.73,0.005541,0.502533,0.682342,0.000029,0.727273,0.049226,0.859242,1.0,0.031243,0.689425,0.481707,green day
1,in the end,0.66,0.010340,0.549139,0.853697,0.000000,0.272727,0.099580,0.801948,0.0,0.052922,0.434371,0.376016,linkin park
2,seven nation army,0.76,0.008202,0.746707,0.462888,0.448345,0.000000,0.250333,0.766734,1.0,0.084166,0.511233,0.329268,the white stripes
3,by the way,0.74,0.026505,0.456940,0.970940,0.003561,0.000000,0.093426,0.838352,1.0,0.113709,0.505303,0.201220,red hot chili peppers
4,how you remind me,0.56,0.000957,0.452888,0.766517,0.000000,0.909091,0.104707,0.835204,1.0,0.033262,0.709856,0.583333,nickelback


**Buat matriks fitur**

In [ ]:
song_features=song_df_normalized.set_index("song_name")
song_features.drop(["artist_name"],axis=1,inplace=True)
song_features.head()

,song_popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,audio_valence
song_name,,,,,,,,,,,,
boulevard of broken dreams,0.73,0.005541,0.502533,0.682342,0.000029,0.727273,0.049226,0.859242,1.0,0.031243,0.689425,0.481707
in the end,0.66,0.010340,0.549139,0.853697,0.000000,0.272727,0.099580,0.801948,0.0,0.052922,0.434371,0.376016
seven nation army,0.76,0.008202,0.746707,0.462888,0.448345,0.000000,0.250333,0.766734,1.0,0.084166,0.511233,0.329268
by the way,0.74,0.026505,0.456940,0.970940,0.003561,0.000000,0.093426,0.838352,1.0,0.113709,0.505303,0.201220
how you remind me,0.56,0.000957,0.452888,0.766517,0.000000,0.909091,0.104707,0.835204,1.0,0.033262,0.709856,0.583333


In [ ]:
song_features.info()

<class 'pandas.core.frame.DataFrame'>
Index: 13982 entries, boulevard of broken dreams to up to me
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   song_popularity   13982 non-null  float64
 1   acousticness      13982 non-null  float64
 2   danceability      13982 non-null  float64
 3   energy            13982 non-null  float64
 4   instrumentalness  13982 non-null  float64
 5   key               13982 non-null  float64
 6   liveness          13982 non-null  float64
 7   loudness          13982 non-null  float64
 8   audio_mode        13982 non-null  float64
 9   speechiness       13982 non-null  float64
 10  tempo             13982 non-null  float64
 11  audio_valence     13982 non-null  float64
dtypes: float64(12)
memory usage: 1.4+ MB


In [ ]:
song_features.tail()

,song_popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,audio_valence
song_name,,,,,,,,,,,,
let it breathe,0.60,0.896586,0.506586,0.150241,0.000066,1.000000,0.102656,0.561569,1.0,0.036982,0.470328,0.304878
answers,0.60,0.768072,0.501520,0.160262,0.000001,1.000000,0.096503,0.611850,0.0,0.031987,0.389100,0.269309
sudden love (acoustic),0.23,0.850401,0.728470,0.324602,0.000000,0.000000,0.117014,0.657845,1.0,0.037726,0.538689,0.290650
gentle on my mind,0.55,0.948795,0.494428,0.325604,0.015747,0.272727,0.110860,0.662850,1.0,0.034857,0.437702,0.328252
up to me,0.60,0.914659,0.648430,0.380718,0.000255,0.363636,0.095477,0.668550,1.0,0.032094,0.377562,0.590447


**Gunakan algo KNN berbasis kesamaan kosinus untuk menghitung jarak minimum dan menemukan lagu yang serupa**

In [ ]:
from scipy.sparse import csr_matrix

song_features_csr = csr_matrix(song_features.values)

In [ ]:
song_features_csr

<13982x12 sparse matrix of type '<class 'numpy.float64'>'
	with 155885 stored elements in Compressed Sparse Row format>

In [ ]:
song_features.values

array([[0.73      , 0.00554115, 0.50253293, ..., 0.03124336, 0.68942464,
        0.48170732],
       [0.66      , 0.01034035, 0.5491388 , ..., 0.05292242, 0.43437136,
        0.37601626],
       [0.76      , 0.0082018 , 0.74670719, ..., 0.08416578, 0.51123317,
        0.32926829],
       ...,
       [0.23      , 0.85040145, 0.72847011, ..., 0.03772582, 0.53868883,
        0.29065041],
       [0.55      , 0.94879513, 0.49442756, ..., 0.03485654, 0.4377017 ,
        0.32825203],
       [0.6       , 0.91465855, 0.64842958, ..., 0.03209352, 0.37756172,
        0.59044715]])

In [ ]:
from sklearn.neighbors import NearestNeighbors

model_nn = NearestNeighbors(metric='cosine',algorithm='brute')
model_nn.fit(song_features_csr)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [ ]:
temp = song_features.copy()
temp.reset_index(inplace=True)
songsearch = 'footloose'
songsearch = songsearch.lower()
song_index = temp.index[temp['song_name'] ==songsearch].tolist()[0]
print(song_index)

print(song_features.index[song_index])


distances,indices = model_nn.kneighbors(X = song_features.iloc[song_index,:].values.reshape(1,-1), n_neighbors=6)

for i in range(0, len(distances.flatten())):
    if i == 0:
        print("Recommendation for ",song_features.index[song_index],"are: ")
    else:
        print(i,": ",song_features.index[indices.flatten()[i]], "| distance= ",distances.flatten()[i])

722
footloose
Recommendation for  footloose are: 
1 :  song about you | distance=  0.0020305401383119426
2 :  levitate | distance=  0.003666470437816205
3 :  misery business | distance=  0.00403479925271677
4 :  umbrella | distance=  0.0045501133174327135
5 :  paris | distance=  0.0052078547937703945


**Temukan artis lagu**

In [ ]:
songgname = 'footloose'
songgname = songgname.lower()
songg = song_info[song_info['song_name'] == songgname]
#songg['artist_name'].unique().tolist()[0]
songg

,song_name,artist_name,album_names,playlist
778,footloose,kenny loggins,footloose (15th anniversary collectors' edition),80s smash hits


**Temukan album lagu**

In [ ]:
songgname = 'red red wine'
songgname = songgname.lower()
songg = song_info[song_info['song_name'] == songgname]
#songg['album_names'].unique().tolist()
songg

,song_name,artist_name,album_names,playlist
2304,red red wine,ub40,the very best of,all out 80s


**Temukan lagu artis**

In [ ]:
artistname = 'aretha franklin'
artistname = artistname.lower()
artist = song_df[song_df['artist_name'] == artistname]
artist['song_name'].unique().tolist()
#artist

['freeway of love - single mix',
 'i say a little prayer',
 'think',
 'respect',
 'chain of fools',
 'respect - remastered',
 'son of a preacher man',
 '(you make me feel like) a natural woman']

**Membagi df utama sesuai daftar putar**

In [ ]:
song = 'i say a little prayer'
song = song.lower()

In [ ]:
playlist = song_info[song_info['song_name'] == song]
x = playlist['playlist'].unique().tolist()
x[0]

'all out 60s'

In [ ]:
temp_df = song_df.copy()
temp_df[['artist_name','playlist']] = song_info[['artist_name','playlist']]
temp_df = temp_df.drop_duplicates(subset=['song_name','artist_name'])


temp_df = temp_df.reset_index()
temp_df = temp_df.drop('index',axis=1)

temp_df[temp_df['playlist'] == x[0]]

,song_name,song_popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,audio_mode,speechiness,tempo,audio_valence,artist_name,playlist
2039,hypnotize,69,0.1760,0.895,0.694,0.000000,9,0.2920,-4.384,1,0.2750,93.944,0.632,marvin gaye,all out 60s
2040,my love is your love,58,0.0449,0.770,0.471,0.000004,7,0.0669,-9.578,1,0.1560,82.515,0.474,the rolling stones,all out 60s
2043,step by step,63,0.0274,0.787,0.630,0.000007,0,0.1610,-12.295,1,0.0609,125.347,0.882,the animals,all out 60s
2044,nothing compares 2 u,58,0.0425,0.511,0.574,0.000023,5,0.1050,-7.016,1,0.0273,119.917,0.161,the temptations,all out 60s
2045,californication,82,0.0021,0.592,0.767,0.001650,9,0.1270,-2.788,0,0.0270,96.483,0.328,jackie wilson,all out 60s
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2123,dark times,31,0.1060,0.585,0.421,0.000010,7,0.1400,-9.593,1,0.0707,132.986,0.240,bob dylan,all out 60s
2124,i miss the old u,74,0.3100,0.716,0.812,0.000000,7,0.5510,-5.233,1,0.3440,133.047,0.385,the delfonics,all out 60s
2130,she don't (feat. ty dolla $ign),76,0.2680,0.650,0.478,0.000000,7,0.1770,-7.512,0,0.0500,99.979,0.147,aretha franklin,all out 60s
2131,think about me,70,0.0607,0.547,0.294,0.786000,7,0.0933,-10.202,1,0.0482,105.027,0.110,bob dylan,all out 60s
